In [20]:
import pandas as pd
import numpy as np

# Predicting 2018-19 winners and teams that have relegated using linear regression
# model from previous year data

# This model cannot be used in real world as teams performace depends upon the players and managers and
# not on previous year datas

In [2]:
data = pd.read_csv("/content/drive/MyDrive/Project_reviews/FMEL_Dataset.csv")
data.head(10)

,id,season,division,round,localTeam,visitorTeam,localGoals,visitorGoals,date,timestamp
0,1,1970-71,1,1,Atletico de Bilbao,Barcelona,1,1,12/09/1970,21938400
1,2,1970-71,1,1,Las Palmas,Atletico de Madrid,1,1,12/09/1970,21938400
2,3,1970-71,1,1,Real Madrid,Valencia,2,0,12/09/1970,21938400
3,4,1970-71,1,1,Celta de Vigo,Sporting de Gijon,2,0,13/09/1970,22024800
4,5,1970-71,1,1,Elche,Granada,1,1,13/09/1970,22024800
5,6,1970-71,1,1,Espanol,Sevilla,0,1,13/09/1970,22024800
6,7,1970-71,1,1,Sabadell,Real Sociedad,0,0,13/09/1970,22024800
7,8,1970-71,1,1,Zaragoza,CD Malaga,0,0,13/09/1970,22024800
8,9,1970-71,1,2,Valencia,Las Palmas,5,1,19/09/1970,22543200
9,10,1970-71,1,2,Atletico de Madrid,Atletico de Bilbao,2,0,20/09/1970,22629600


In [3]:
data = data.drop(["id","timestamp"],axis=1)
data.head(5)

,season,division,round,localTeam,visitorTeam,localGoals,visitorGoals,date
0,1970-71,1,1,Atletico de Bilbao,Barcelona,1,1,12/09/1970
1,1970-71,1,1,Las Palmas,Atletico de Madrid,1,1,12/09/1970
2,1970-71,1,1,Real Madrid,Valencia,2,0,12/09/1970
3,1970-71,1,1,Celta de Vigo,Sporting de Gijon,2,0,13/09/1970
4,1970-71,1,1,Elche,Granada,1,1,13/09/1970


In [4]:
condition = [(data["localGoals"] > data["visitorGoals"]),(data["localGoals"] < data["visitorGoals"]),
             (data["localGoals"] == data["visitorGoals"])]
value = ["local","visitor","draw"]

data["result"] = np.select(condition,value)

data.head(5)

,season,division,round,localTeam,visitorTeam,localGoals,visitorGoals,date,result
0,1970-71,1,1,Atletico de Bilbao,Barcelona,1,1,12/09/1970,draw
1,1970-71,1,1,Las Palmas,Atletico de Madrid,1,1,12/09/1970,draw
2,1970-71,1,1,Real Madrid,Valencia,2,0,12/09/1970,local
3,1970-71,1,1,Celta de Vigo,Sporting de Gijon,2,0,13/09/1970,local
4,1970-71,1,1,Elche,Granada,1,1,13/09/1970,draw


In [5]:
df = data.merge(pd.get_dummies(data['result']), left_index=True, right_index=True)

df['local_wins_this_season'] = df.groupby(['season','localTeam'])['local'].transform('sum')
df['visitor_wins_this_season'] = df.groupby(['season','visitorTeam'])['visitor'].transform('sum')

print(" ")

In [6]:
visitor_results = (df
                   .groupby(['season', 'visitorTeam'])['visitor']
                   .sum()
                   .reset_index()
                   .rename(columns={'visitorTeam': 'team',
                                    'visitor': 'visitor_wins'}))

local_results = (df
                 .groupby(['season', 'localTeam'])['local']
                 .sum()
                 .reset_index()
                 .rename(columns={'localTeam': 'team',
                                  'local': 'local_wins'}))

wins_per_season = visitor_results.merge(local_results, on=['season', 'team'])

wins_per_season['total_wins'] = wins_per_season['visitor_wins'] + wins_per_season['local_wins']

wins_per_season.head(5)

,season,team,visitor_wins,local_wins,total_wins
0,1970-71,Atletico de Bilbao,3,11,14
1,1970-71,Atletico de Madrid,4,13,17
2,1970-71,Barcelona,8,11,19
3,1970-71,Betis,5,15,20
4,1970-71,Burgos,3,16,19


In [7]:
total_wins_sorted_desc = (wins_per_season
                          .groupby(['team'])['total_wins']
                          .sum()
                          .sort_values(ascending=False)
                          .reset_index()['team'])

wins_per_season_pivot = (wins_per_season
                         .pivot_table(index='team',
                                      columns='season',
                                      values='total_wins')
                         .fillna(0)
                         .reindex(total_wins_sorted_desc))

In [8]:
# total wins of each teams for each season

print(wins_per_season_pivot)

season                     1970-71  1971-72  1972-73  1973-74  1974-75  \
team                                                                     
Real Madrid                   17.0     19.0     17.0     13.0     20.0   
Barcelona                     19.0     17.0     18.0     21.0     15.0   
Atletico de Madrid            17.0     14.0     20.0     18.0     11.0   
Valencia                      18.0     19.0     12.0     13.0     12.0   
Sevilla                       13.0      9.0     16.0     15.0     22.0   
...                            ...      ...      ...      ...      ...   
Ontinyent                      8.0      0.0      0.0      0.0      0.0   
Universidad de Las Palmas      0.0      0.0      0.0      0.0      0.0   
Lorca FC                       0.0      0.0      0.0      0.0      0.0   
Moscardo                       6.0      0.0      0.0      0.0      0.0   
Mollerussa                     0.0      0.0      0.0      0.0      0.0   

season                     1975-76  1

In [9]:
df = pd.DataFrame(wins_per_season_pivot.to_records())

df

,team,1970-71,1971-72,1972-73,1973-74,1974-75,1975-76,1976-77,1977-78,1978-79,...,2008-09,2009-10,2010-11,2011-12,2012-13,2013-14,2014-15,2015-16,2016-17,2017-18
0,Real Madrid,17.0,19.0,17.0,13.0,20.0,20.0,12.0,22.0,16.0,...,25.0,31.0,29.0,32.0,26.0,27.0,30.0,28.0,29.0,22.0
1,Barcelona,19.0,17.0,18.0,21.0,15.0,18.0,18.0,16.0,16.0,...,27.0,31.0,30.0,28.0,32.0,27.0,30.0,29.0,28.0,28.0
2,Atletico de Madrid,17.0,14.0,20.0,18.0,11.0,18.0,19.0,16.0,14.0,...,20.0,13.0,17.0,15.0,23.0,28.0,23.0,28.0,23.0,23.0
3,Valencia,18.0,19.0,12.0,13.0,12.0,12.0,13.0,16.0,14.0,...,18.0,21.0,21.0,17.0,19.0,13.0,22.0,11.0,13.0,22.0
4,Sevilla,13.0,9.0,16.0,15.0,22.0,13.0,11.0,13.0,12.0,...,21.0,19.0,17.0,13.0,14.0,18.0,23.0,14.0,21.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,Ontinyent,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
111,Universidad de Las Palmas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112,Lorca FC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
113,Moscardo,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
col = [i for i in range(1970,2018)]
col_ind = [j for j in range(1,49)]
print(len(col))
print(len(col_ind))

48
48


In [11]:
old_names = df.columns[col_ind]
df.rename(columns=dict(zip(old_names, col)), inplace=True)
df

,team,1970,1971,1972,1973,1974,1975,1976,1977,1978,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Real Madrid,17.0,19.0,17.0,13.0,20.0,20.0,12.0,22.0,16.0,...,25.0,31.0,29.0,32.0,26.0,27.0,30.0,28.0,29.0,22.0
1,Barcelona,19.0,17.0,18.0,21.0,15.0,18.0,18.0,16.0,16.0,...,27.0,31.0,30.0,28.0,32.0,27.0,30.0,29.0,28.0,28.0
2,Atletico de Madrid,17.0,14.0,20.0,18.0,11.0,18.0,19.0,16.0,14.0,...,20.0,13.0,17.0,15.0,23.0,28.0,23.0,28.0,23.0,23.0
3,Valencia,18.0,19.0,12.0,13.0,12.0,12.0,13.0,16.0,14.0,...,18.0,21.0,21.0,17.0,19.0,13.0,22.0,11.0,13.0,22.0
4,Sevilla,13.0,9.0,16.0,15.0,22.0,13.0,11.0,13.0,12.0,...,21.0,19.0,17.0,13.0,14.0,18.0,23.0,14.0,21.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,Ontinyent,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
111,Universidad de Las Palmas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112,Lorca FC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
113,Moscardo,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# Linear refression model to check score for the next season

from scipy import stats

def myfunc(x):
  return slope * x + intercept

x = col
lst=[]

for i in range(0,115):
  y = list(df.loc[i,])
  y.pop(0)

  slope, intercept, r, p, std_err = stats.linregress(x, y)
  speed = myfunc(2018)
  lst.append(speed)

print(lst)


[27.632092198581574, 28.1409574468085, 20.204787234042527, 18.995567375886537, 18.18971631205676, 14.400709219858157, 14.782801418439716, 15.607269503546092, 14.938829787234042, 14.272163120567377, 15.478723404255305, 14.695921985815616, 15.031914893617, 12.74202127659575, 11.476063829787236, 14.442375886524843, 13.824468085106389, 11.785460992907803, 11.008865248226954, 7.1817375886524815, 13.343971631205704, 4.078900709219852, 10.338652482269495, 6.848404255319167, 6.184397163120565, 20.33421985815596, 2.6533687943262976, 6.550531914893611, 12.24024822695037, 0.00886524822692536, 15.241134751772961, 10.430851063829778, 13.075354609929093, 4.356382978723417, 12.965425531914889, 7.069148936170215, 16.81826241134752, 1.1179078014184256, -3.590425531914889, 14.982269503546036, 2.8874113475177126, 9.840425531914889, 14.43351063829789, 14.125, 1.4166666666666856, 8.273049645390074, 5.215425531914903, 8.533687943262407, 1.695035460992898, 11.497340425531888, -4.252659574468112, 4.0523049645

In [13]:
# Winner predicting using linear regression

ind = lst.index(max(lst))
print(df.team[ind])

# winner was predicted correctly

Barcelona


In [14]:
# Teams with regression score less than 1, this shows teams which are predicted not win a single game
# These teams might be relegated

reg = [df.team[lst.index(i)] for i in lst if i<1]


In [15]:
teams_2018_19 = ["Alaves","Athletic Bilbao","Atletico Madrid","Barcelona","Celta Vigo","Deportivo La Coruña","Eibar","Espanyol","Getafe","Girona","Universidad de Las Palmas",
                 "Leganes","Levante","CD Malaga","Real Betis","Real Madrid","Real Sociedad","Sevilla","Valencia","Valladolid","Villarreal"]


In [16]:
# teams relegated in 2018_19 season

for i in reg:
  if i in teams_2018_19:
    print(i)

# 2 out of 3 teams were predicted

CD Malaga
Universidad de Las Palmas
